# Upload Spotify Tracks DB

Upload the dataset to DB using the datatset https://www.kaggle.com/datasets/zaheenhamidani/ultimate-spotify-tracks-db

We have the database credentials saved in the `db_config.txt` file. Please reference `db_config_example.txt` for format. 

In [7]:
def read_db_config(filename="db_config.txt"):
    config = {}
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue  # Skip empty or commented lines
            key, sep, value = line.partition("=")
            if sep:
                config[key.strip()] = value.strip()
    return config

In [8]:
import pandas as pd

CSV = "../data/SpotifyFeatures.csv"
KEEP = ["track_id", "track_name", "artist_name", "valence", "tempo", "popularity"]
df = pd.read_csv(CSV, usecols=KEEP)

df.rename(
    columns={
        "track_id": "spotify_id",
        "track_name": "track_name",
        "artist_name": "artist",
    },
    inplace=True,
)

df = df[["spotify_id", "track_name", "artist", "valence", "tempo", "popularity"]]
df.dropna(subset=["spotify_id", "track_name"], inplace=True)

Noted that in this code, we uses password to authenticate, and MySQL >= 9 no longer support the native password plugin. 

You can use
```sql
SELECT user, host, plugin from mysql.user WHERE plugin='mysql_native_password';
```
on the database to test which accounts have this plugin loaded.

In [9]:
import mysql.connector as mc
import pandas as pd

cfg = read_db_config()

DB = dict(
    host=cfg.get("DB_HOST"),
    user=cfg.get("DB_USER"),
    password=cfg.get("DB_PASS"),
    database=cfg.get("DB_NAME"),
    allow_local_infile=True,
)
conn = mc.connect(**DB)
cur = conn.cursor()
cur.executemany(
    """
    INSERT IGNORE INTO Song
      (spotify_id, track_name, artist, valence, tempo, popularity)
    VALUES (%s,%s,%s,%s,%s,%s)
""",
    df.values.tolist(),
)
conn.commit()
print("Inserted", cur.rowcount, "rows into Song")
cur.close()
conn.close()

Inserted 0 rows into Song
